In [1]:
# Importing the MDAnalysis package.
import MDAnalysis as mda

# Creating the Universe object.
u = mda.Universe('PDB_tubuline.pdb', 'trajectoire_tubuline.dcd')

# Create a selection of atoms to work with.
ag = u.atoms.select_atoms('backbone')

# Select all solvent within a set distance from protein atoms.
ag = u.select_atoms('resname SOL and around 5.0 protein')

# Select all heavy atoms in the first 20 residues.
ag = u.select_atoms('resid 1:20 and not prop mass < 10.0')

# Use a preexisting AtomGroup as part of another selection.
sel1 = u.select_atoms('name N and not resname MET')
sel2 = u.select_atoms('around 2.5 group Nsel', Nsel=sel1)

# Perform a selection on another AtomGroup.
sel1 = u.select_atoms('around 5.0 protein')
sel2 = sel1.select_atoms('type O')

FileNotFoundError: [Errno 2] No such file or directory: 'PDB_tubuline.pdb'